In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [11]:
url = 'https://www.ikea.com/us/en/p/vimle-sofa-orrsta-black-blue-s59284761/'
response = requests.get(url)
response

<Response [200]>

In [12]:
soup = BeautifulSoup(response.text, "html.parser")
brand = soup.findAll("span", {"class":"product-pip__name"})[0].contents[0]
price = soup.findAll("span", {"class":"product-pip__price__value"})[0].contents[0]
dimensions = soup.findAll("dd", {"class":"product-pip__definition-list-item"})

#copying all dimensions for now, we'll figure out on what values are important in dimensions
d_list = []
for i in range(len(dimensions)):
    d_list.append(dimensions[i].contents[0])
    
imgLink = soup.findAll("img", {"class":""})[0]['src']

brand, price, d_list, imgLink

# pip__name - Name of company
# normal-font range__text-rtl - product Description
# pip-price-component - Price obviously
# pip__definition-list-item (Every second) - Width, Depth, Height

('VIMLE',
 '$599.00',
 ['32 5/8 "',
  '26 3/4 "',
  '94 7/8 "',
  '38 5/8 "',
  '2 3/8 "',
  '5 7/8 "',
  '26 3/4 "',
  '83 1/8 "',
  '21 5/8 "',
  '18 7/8 "'],
 'https://www.ikea.com/PIAimages/0514250_PE639376_S5.JPG?f=s')

In [29]:
#Contains all dimension-names from product description
dimensions_desc = soup.findAll("dt", {"class":"product-pip__definition-list-item"})
print(dimensions_desc)

#Contains all dimension-values from product description
dimensions = soup.findAll("dd", {"class":"product-pip__definition-list-item"})
dimensions

#Finding All height-related dimensions
height_index = []
for dim in range(0, len(dimensions_desc)):
    if("height" in dimensions_desc[dim].contents[0].lower()):
        height_index.append(dim)

height_index

#Finding max of given heights. Seemed in sync with most of the images 
#with minute variations in the exceptions.
heights = []
for index in height_index:
    heights.append(dimensions[index].contents[0])
    
height = max(heights)

width_index = []
for dim in range(0, len(dimensions_desc)):
    if("width" in dimensions_desc[dim].contents[0].lower()):
        width_index.append(dim)

width_index

widths = []
for index in width_index:
    widths.append(dimensions[index].contents[0])
    
width = max(widths)

depth_index = []
for dim in range(0, len(dimensions_desc)):
    if("depth" in dimensions_desc[dim].contents[0].lower()):
        depth_index.append(dim)

depth_index

depths = []
for index in depth_index:
    depths.append(dimensions[index].contents[0])
    
depth = max(depths)
depth


    

[<dt class="product-pip__definition-list-item">Height including back cushions: </dt>, <dt class="product-pip__definition-list-item">Backrest height: </dt>, <dt class="product-pip__definition-list-item">Width: </dt>, <dt class="product-pip__definition-list-item">Depth: </dt>, <dt class="product-pip__definition-list-item">Height under furniture: </dt>, <dt class="product-pip__definition-list-item">Armrest width: </dt>, <dt class="product-pip__definition-list-item">Armrest height: </dt>, <dt class="product-pip__definition-list-item">Seat width: </dt>, <dt class="product-pip__definition-list-item">Seat depth: </dt>, <dt class="product-pip__definition-list-item">Seat height: </dt>]


'38 5/8 "'

[<dd class="product-pip__definition-list-item">31 1/2 "</dd>,
 <dd class="product-pip__definition-list-item">94 7/8 "</dd>,
 <dd class="product-pip__definition-list-item">38 5/8 "</dd>,
 <dd class="product-pip__definition-list-item">1 5/8 "</dd>,
 <dd class="product-pip__definition-list-item">5 7/8 "</dd>,
 <dd class="product-pip__definition-list-item">25 5/8 "</dd>,
 <dd class="product-pip__definition-list-item">83 1/8 "</dd>,
 <dd class="product-pip__definition-list-item">21 5/8 "</dd>,
 <dd class="product-pip__definition-list-item">17 3/4 "</dd>]